In [1]:
import pandas as pd  
import numpy as np
import xgboost
from sklearn.model_selection import train_test_split
import time
from matplotlib import pyplot as plt
import seaborn as sns       
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
orders = pd.read_csv("E:/Masters Data/Fall2022/DataMining/Dataset/orders.csv/orders.csv")
order_products_train = pd.read_csv("E:/Masters Data/Fall2022/DataMining/Dataset/order_products__train.csv/order_products__train.csv")
order_products_prior = pd.read_csv("E:/Masters Data/Fall2022/DataMining/Dataset/order_products__prior.csv/order_products__prior.csv")
products = pd.read_csv("E:/Masters Data/Fall2022/DataMining/Dataset/products.csv/products.csv")
aisles = pd.read_csv("E:/Masters Data/Fall2022/DataMining/Dataset/aisles.csv/aisles.csv")
departments = pd.read_csv("E:/Masters Data/Fall2022/DataMining/Dataset/departments.csv/departments.csv")

In [3]:
orders.dropna()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [4]:
df1 = pd. merge(products,aisles, on="aisle_id" )
df2 = pd.merge(products,departments, on="department_id")
df3 = pd.merge(order_products_train,products)
df4 = pd.merge(order_products_train,orders, on ="order_id")

In [5]:
df_1 = pd.merge(df1,df2)
df= pd.merge(df_1,df3)

In [6]:
df3 = pd.merge(order_products_train,df)
df3.shape


(1384617, 9)

In [7]:
df4= pd.merge(df3,orders)

In [8]:
df4.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0


In [9]:
df4.shape

(1384617, 15)

In [10]:
OrderProduct = pd.merge(orders,order_products_prior, 
                        on='order_id', 
                        how='inner')

In [11]:
TotalOrder = OrderProduct.groupby('user_id')['order_number'].max()
TotalOrder = TotalOrder.to_frame('Ordercount')
TotalOrder = TotalOrder.reset_index()
TotalOrder.head()

,user_id,Ordercount
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [12]:
Totalproduct = df4.groupby('product_id')['order_id'].count()
Totalproduct = Totalproduct.reset_index()
Totalproduct = Totalproduct.rename(columns = {'order_id':'Frequency'})
Totalproduct = Totalproduct.sort_values('Frequency', ascending=False)
Totalproduct = Totalproduct.reset_index(drop=True)
Totalproduct = Totalproduct.iloc[:100]
Totalproduct.head()

,product_id,Frequency
0,24852,18726
1,13176,15480
2,21137,10894
3,21903,9784
4,47626,8135


In [13]:
Productlist = Totalproduct['product_id']
Productlist[0:10]

0    24852
1    13176
2    21137
3    21903
4    47626
5    47766
6    47209
7    16797
8    26209
9    27966
Name: product_id, dtype: int64

In [14]:
products_list = df4[df4['product_id'].isin(Productlist)]
products_list.shape

(314227, 15)

In [15]:
marketbasket = pd.pivot_table(products_list,columns='product_name', 
                                    values='reordered', 
                                    index='order_id')
marketbasket = marketbasket.reset_index().fillna(0)
marketbasket =marketbasket.set_index('order_id')

In [16]:
def change(p):
    if p <= 0:
        return 0
    if p >= 1:
        return 1
    
marketbasket = marketbasket.applymap(change)
marketbasket.head()

product_name,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Blueberries,Boneless Skinless Chicken Breasts,Broccoli Crown,Bunched Cilantro,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [17]:
basksetselection = marketbasket[:15000]
basksetselection

product_name,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Blueberries,Boneless Skinless Chicken Breasts,Broccoli Crown,Bunched Cilantro,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539209,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539218,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
539236,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Items = apriori(basksetselection, min_support=0.02, use_colnames=True)
Items.head()

C:\Users\Admin\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.026933,(Asparagus)
1,0.142067,(Bag of Organic Bananas)
2,0.174333,(Banana)
3,0.027933,(Honeycrisp Apple)
4,0.063800,(Large Lemon)


In [19]:
associationRules = association_rules(Items, metric='lift', min_threshold=0.8)
associationRules.sort_values('lift')
associationRules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Hass Avocado),(Bag of Organic Bananas),0.062667,0.142067,0.020267,0.323404,2.276426,0.011364,1.268015
1,(Bag of Organic Bananas),(Organic Hass Avocado),0.142067,0.062667,0.020267,0.142656,2.276426,0.011364,1.093299
2,(Bag of Organic Bananas),(Organic Strawberries),0.142067,0.090267,0.026467,0.186298,2.063857,0.013643,1.118017
3,(Organic Strawberries),(Bag of Organic Bananas),0.090267,0.142067,0.026467,0.293205,2.063857,0.013643,1.213837
4,(Banana),(Organic Avocado),0.174333,0.069067,0.020800,0.119312,1.727485,0.008759,1.057052


In [20]:
output = df4.to_csv(r'C:\Users\Admin\Data Mining\output.csv')

### Second Part

In [21]:
df4 = df4.iloc[0:100000]

In [22]:
df4.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0


In [23]:
df4 = df4[['add_to_cart_order','order_number','order_dow','order_hour_of_day','reordered']]
df4.head()

,add_to_cart_order,order_number,order_dow,order_hour_of_day,reordered
0,1,4,4,10,1
1,2,4,4,10,1
2,3,4,4,10,0
3,4,4,4,10,0
4,5,4,4,10,1


In [24]:
df4.isnull().any()

add_to_cart_order    False
order_number         False
order_dow            False
order_hour_of_day    False
reordered            False
dtype: bool

In [25]:
df4.replace(np.nan,0,inplace=True)

In [26]:
df4.dtypes

add_to_cart_order    int64
order_number         int64
order_dow            int64
order_hour_of_day    int64
reordered            int64
dtype: object

In [27]:
x=df4.iloc[:,0:3].values
x

array([[ 1,  4,  4],
       [ 2,  4,  4],
       [ 3,  4,  4],
       ...,
       [ 4, 21,  5],
       [ 5, 21,  5],
       [ 6, 21,  5]], dtype=int64)

In [28]:
y=df4.iloc[:,-1].values
y

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [29]:
from sklearn.preprocessing import LabelEncoder

In [30]:
lb=LabelEncoder()

In [31]:
x

array([[ 1,  4,  4],
       [ 2,  4,  4],
       [ 3,  4,  4],
       ...,
       [ 4, 21,  5],
       [ 5, 21,  5],
       [ 6, 21,  5]], dtype=int64)

In [32]:
x[:,0]= lb.fit_transform(x[:,0])
x[:,1] = lb.fit_transform(x[:,1])
x[:,2] =lb.fit_transform(x[:,2])







In [33]:
x

array([[ 0,  0,  4],
       [ 1,  0,  4],
       [ 2,  0,  4],
       ...,
       [ 3, 17,  5],
       [ 4, 17,  5],
       [ 5, 17,  5]], dtype=int64)

In [34]:
y[:]=lb.fit_transform(y[:])

In [35]:
y

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [36]:
from sklearn.model_selection import train_test_split 

In [37]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0) 

In [38]:
x_train

array([[30, 18,  1],
       [ 0,  2,  6],
       [26, 38,  0],
       ...,
       [ 4,  8,  3],
       [ 7,  2,  2],
       [ 0,  2,  2]], dtype=int64)

In [39]:
x_test

array([[ 2, 29,  3],
       [ 6,  2,  1],
       [ 1,  2,  1],
       ...,
       [ 8, 24,  1],
       [ 2, 33,  0],
       [ 7,  9,  6]], dtype=int64)

In [40]:
y_train

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [41]:
y_test

array([0, 0, 1, ..., 0, 1, 1], dtype=int64)

In [42]:
from sklearn.preprocessing import MinMaxScaler

In [43]:
sc=MinMaxScaler()

In [44]:
x_train=sc.fit_transform(x_train)

In [45]:
x_test=sc.transform(x_test)

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,roc_auc_score
from sklearn.model_selection import GridSearchCV

In [47]:
list = []

In [48]:
dict = {}
logr=LogisticRegression()
logr.fit(x_train,y_train)
y_pred=logr.predict(x_test)
dict['Algorithm Model'] = 'Logistic Regression'
dict['Accuracy'] = accuracy_score(y_test,y_pred)
dict['Recall'] = recall_score(y_test,y_pred)
dict['Precision'] = precision_score(y_test,y_pred)
dict['F1 score'] = f1_score(y_test,y_pred)
dict['roc_auc'] = roc_auc_score(y_test,y_pred)
list.append(dict)

In [49]:
dict1 = {}
dt=DecisionTreeClassifier(criterion='entropy',max_depth=5,max_features='auto',random_state=123,splitter='best')
dt.fit(x_train,y_train)
y_pred1=dt.predict(x_test)
dict1['Algorithm Model'] = 'Decision Tree Classisifer'
dict1['Accuracy'] = accuracy_score(y_test,y_pred1)
dict1['Recall'] = recall_score(y_test,y_pred1)
dict1['Precision'] = precision_score(y_test,y_pred1)
dict1['F1 score'] = f1_score(y_test,y_pred1)
dict1['roc_auc'] = roc_auc_score(y_test,y_pred1)
list.append(dict1)


In [50]:
dict2 = {}
rf=RandomForestClassifier(n_estimators=1000,criterion='entropy',random_state=0)
rf.fit(x_train,y_train)
y_pred2=rf.predict(x_test)
dict2['Algorithm Model'] = 'Random Forest'
dict2['Accuracy'] = accuracy_score(y_test,y_pred2)
dict2['Recall'] = recall_score(y_test,y_pred2)
dict2['Precision'] = precision_score(y_test,y_pred2)
dict2['F1 score'] = f1_score(y_test,y_pred2)
dict2['roc_auc'] = roc_auc_score(y_test,y_pred2)
list.append(dict2)


In [51]:
dict3 = {}
knn=KNeighborsClassifier(n_neighbors=5,p=2)
knn.fit(x_train,y_train)
y_pred3=knn.predict(x_test)
dict3['Algorithm Model'] = 'KNN'
dict3['Accuracy'] = accuracy_score(y_test,y_pred3)
dict3['Recall'] = recall_score(y_test,y_pred3)
dict3['Precision'] = precision_score(y_test,y_pred3)
dict3['F1 score'] = f1_score(y_test,y_pred3)
dict3['roc_auc'] = roc_auc_score(y_test,y_pred3)
list.append(dict3)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [52]:
table = pd.DataFrame(list)
table

,Algorithm Model,Accuracy,Recall,Precision,F1 score,roc_auc
0,Logistic Regression,0.63905,0.871203,0.649428,0.744143,0.579187
1,Decision Tree Classisifer,0.63465,0.803817,0.662110,0.726114,0.591028
2,Random Forest,0.62825,0.752033,0.670812,0.709104,0.596331
3,KNN,0.60625,0.730041,0.655563,0.690801,0.574329
